# Tarea 02 - Análisis de datos geoespaciales mediante Fiona y Shapely

José P. Barrantes <br>
Ricardo Corrales Barquero - B32090

### Librerías

In [ ]:
from owslib.wfs import WebFeatureService
from geojson import dump
import requests
import fiona
from shapely.geometry import shape, mapping
import rtree

## Obtención de datos
En el [sitio web del SNIT](https://www.snitcr.go.cr/) podemos obtener la url de los WFS que necesitaremos. Una vez las tenemos, podemos consultar las capas que vamos a utilizar en esta pŕactica.

In [2]:
wfs_cartografia_1_5mil = WebFeatureService(url='https://geos.snitcr.go.cr/be/IGN_5/wfs?', version='1.1.0')
list(wfs_cartografia_1_5mil.contents)

['IGN_5:forestal2017_5k',
 'IGN_5:indice_5000',
 'IGN_5:cultivos2017_5k',
 'IGN_5:curvas_5000',
 'IGN_5:delimitacion2017_5k',
 'IGN_5:edificaciones2017_5k',
 'IGN_5:hidrografia_5000',
 'IGN_5:limitecantonal_5k',
 'IGN_5:limitedistrital_5k',
 'IGN_5:limiteprovincial_5k',
 'IGN_5:linea_costa_5000',
 'IGN_5:pastos2017_5k',
 'IGN_5:urbano_5000',
 'IGN_5:vias_5000']

De acá, la capa 'IGN_5:limitecantonal_5k' es la que nos resulta de interés. Procedemos a descargar la capa con las divisiones cantonales.

In [5]:
# Solicitud de capa WFS de límite cantonal mediante GET, para retornarse como JSON

# Parámetros de la solicitud
params = dict(service='WFS',
              version='1.1.0', 
              request='GetFeature', 
              typeName='IGN_5:limitecantonal_5k',
              srsName='urn:ogc:def:crs:EPSG::4326',
              outputFormat='json')

# Solicitud
response = requests.get("https://geos.snitcr.go.cr/be/IGN_5/wfs?", params=params)

# Descarga de la respuesta en un archivo GeoJSON

with open('../datos/limite_cantonal.geojson', 'w') as file:
    dump(response.json(), file)

Guardamos esta capa en el archivo 'limite_cantonal.geojson'

Hacemos lo mismo para obtener la información de las carreteras.

In [6]:
wfs_cartografia_1_200mil = WebFeatureService(url='https://geos.snitcr.go.cr/be/IGN_200/wfs?version=1.1.0', version='1.1.0')
list(wfs_cartografia_1_200mil.contents)

['IGN_200:aeropuertointernacional_200k',
 'IGN_200:aerodromos_200k',
 'IGN_200:bordecostarica_200k',
 'IGN_200:cotafotogrametrica_200k',
 'IGN_200:curvas_de_nivel_200k',
 'IGN_200:edificaciones_y_construcciones_200k',
 'IGN_200:embalses_200k',
 'IGN_200:estacionferroviaria_200k',
 'IGN_200:hojas_200_completas',
 'IGN_200:lago_o_laguna_200k',
 'IGN_200:Laguna_Intermitente_1_200mil',
 'IGN_200:lineas_de_costa_200k',
 'IGN_200:muelle_200k',
 'IGN_200:redvial_200k',
 'IGN_200:reddrenaje_200k',
 'IGN_200:salinas_200k',
 'IGN_200:viaferrea_200k']

In [7]:
# Solicitud de capa WFS de red vial mediante GET, para retornarse como JSON

# Parámetros de la solicitud
params = dict(service='WFS',
              version='1.1.0', 
              request='GetFeature', 
              typeName='IGN_200:redvial_200k',
              srsName='urn:ogc:def:crs:EPSG::4326',
              outputFormat='json')

# Solicitud
response = requests.get("https://geos.snitcr.go.cr/be/IGN_200/wfs?version=1.1.0", params=params)

# Descarga de la respuesta en un archivo GeoJSON

with open('../datos/red_vial.geojson', 'w') as file:
    dump(response.json(), file)

La capa de las carreteras la guardamos como 'red_vial.geojson'

## Lectura de los datos con fiona

In [8]:
cantones = fiona.open('../datos/limite_cantonal.geojson')

In [17]:
cantones[0]

{'type': 'Feature',
 'id': '0',
 'properties': OrderedDict([('id', 'limitecantonal_5k.1'),
              ('gmlid', 'limitecantonal_5k.1'),
              ('cod_catalo', '160104'),
              ('cod_canton', 610),
              ('canton', 'Corredores'),
              ('ori_toponi',
               'Tiene su origen en el topónimo del río Corredor, el cual nace en las laderas de la fila Brunqueña'),
              ('area', 623.61),
              ('cod_provin', 6),
              ('provincia', 'Puntarenas'),
              ('version', '20201222')]),
 'geometry': {'type': 'Polygon',
  'coordinates': [[(-82.94161057, 8.42038896),
    (-82.94162692, 8.42039031),
    (-82.9417427, 8.42035319),
    (-82.9420243, 8.42010414),
    (-82.94204546, 8.42009183),
    (-82.94264461, 8.41974344),
    (-82.94293602, 8.41962868),
    (-82.94339878, 8.41934061),
    (-82.94448701, 8.4187649),
    (-82.94482438, 8.4186791),
    (-82.9449041, 8.41861196),
    (-82.94773076, 8.41712069),
    (-82.94789778, 8.417

In [16]:
for i in range(len(cantones)):
    print(cantones[i]['geometry']['type'])

Polygon
MultiPolygon
Polygon
Polygon
Polygon
Polygon
Polygon
Polygon
Polygon
Polygon
Polygon
Polygon
Polygon
Polygon
Polygon
Polygon
Polygon
Polygon
Polygon
Polygon
MultiPolygon
Polygon
Polygon
Polygon
Polygon
Polygon
Polygon
Polygon
Polygon
Polygon
Polygon
Polygon
Polygon
Polygon
Polygon
Polygon
Polygon
Polygon
Polygon
Polygon
Polygon
Polygon
Polygon
Polygon
Polygon
Polygon
Polygon
Polygon
Polygon
Polygon
MultiPolygon
Polygon
Polygon
Polygon
Polygon
Polygon
Polygon
Polygon
Polygon
Polygon
Polygon
Polygon
Polygon
Polygon
Polygon
Polygon
Polygon
Polygon
MultiPolygon
Polygon
Polygon
Polygon
Polygon
Polygon
Polygon
Polygon
Polygon
Polygon
Polygon
Polygon
Polygon
Polygon


<span style="color:red">Eliminar esta parte de geopandas</span>

In [10]:
import geopandas as gpd

In [11]:
aaa = gpd.read_file('../datos/limite_cantonal.geojson')
aaa

C:\Users\ricor\anaconda3\lib\site-packages\geopandas\array.py:85: ShapelyDeprecationWarning: __len__ for multi-part geometries is deprecated and will be removed in Shapely 2.0. Check the length of the `geoms` property instead to get the  number of parts of a multi-part geometry.
  aout[:] = out


,id,gmlid,cod_catalo,cod_canton,canton,ori_toponi,area,cod_provin,provincia,version,geometry
0,limitecantonal_5k.1,limitecantonal_5k.1,160104,610,Corredores,Tiene su origen en el topónimo del río Corredo...,623.61,6,Puntarenas,20201222,"POLYGON ((-82.94161 8.42039, -82.94163 8.42039..."
1,limitecantonal_5k.2,limitecantonal_5k.2,160104,607,Golfito,"Se debe a la forma que presenta el litoral, d...",1752.75,6,Puntarenas,20201222,"MULTIPOLYGON (((-83.46678 8.68886, -83.46677 8..."
2,limitecantonal_5k.3,limitecantonal_5k.3,160104,608,Coto Brus,De origen indígena las palabras Coto (couto) B...,944.24,6,Puntarenas,20201222,"POLYGON ((-82.90501 8.77425, -82.90572 8.77456..."
3,limitecantonal_5k.4,limitecantonal_5k.4,160104,605,Osa,"En recuerdo del cacique Osa, que en los inicio...",1932.70,6,Puntarenas,20201222,"POLYGON ((-83.83946 9.25534, -83.83900 9.25533..."
4,limitecantonal_5k.5,limitecantonal_5k.5,160104,603,Buenos Aires,El nombre se le dio por la brisa constante que...,2382.94,6,Puntarenas,20201222,"POLYGON ((-83.32101 9.38409, -83.32072 9.38405..."
...,...,...,...,...,...,...,...,...,...,...,...
77,limitecantonal_5k.78,limitecantonal_5k.78,160104,215,Guatuso,Dos versiones: 1- Los indigenas vivían cerca d...,752.83,2,Alajuela,20201222,"POLYGON ((-84.80404 10.85084, -84.80379 10.850..."
78,limitecantonal_5k.79,limitecantonal_5k.79,160104,214,Los Chiles,Dos versiones: 1-Por una gran plantación de ch...,1332.71,2,Alajuela,20201222,"POLYGON ((-84.66639 11.07246, -84.60604 11.038..."
79,limitecantonal_5k.80,limitecantonal_5k.80,160104,501,Liberia,El origen del nombre del cantón se remonta a 1...,1442.17,5,Guanacaste,20201222,"POLYGON ((-85.47140 10.96502, -85.47129 10.965..."
80,limitecantonal_5k.81,limitecantonal_5k.81,160104,213,Upala,Voz indígena del náhualt que significa: sobre ...,1592.67,2,Alajuela,20201222,"POLYGON ((-85.24049 11.06368, -85.23930 11.063..."


## Intersección

In [72]:
vias = fiona.open('../datos/red_vial.geojson')

In [73]:
for i in range(len(vias)):
    print(vias[i]['properties']) # Por alguna razón aquí no lo lee bien, no sé por qué
    if vias[i] is not None:
        print(i)

OrderedDict([('origen', 'CR25'), ('categoria', 'CAMINO DE TIERRA'), ('codigo', '130107'), ('num_ruta', None), ('jerarquia', None), ('nombre', None), ('num_carril', None), ('mat_supe', None), ('est_supe', None), ('condi_uso', None), ('administra', None), ('fiabilidad', None), ('longitud', 8744.95001306), ('num_carr', None), ('estac_peaj', None), ('id', 0), ('tipo', None), ('et_id', 0), ('et_source', None), ('fid_', 0), ('entity', None), ('handle', None), ('layer', None), ('lyrfrzn', 0), ('lyrlock', 0), ('lyron', 0), ('lyrvpfrzn', 0), ('lyrhandle', None), ('color', 0), ('entcolor', 0), ('lyrcolor', 0), ('blkcolor', 0), ('linetype', None), ('entlinetyp', None), ('lyrlntype', None), ('blklinetyp', None), ('elevation', 0), ('thickness', 0), ('linewt', 0), ('entlinewt', 0), ('lyrlinewt', 0), ('blklinewt', 0), ('refname', None), ('ltscale', 0), ('extx', 0), ('exty', 0), ('extz', 0), ('docname', None), ('docpath', None), ('doctype', None), ('docver', None)])
0


TypeError: 'NoneType' object is not subscriptable

In [ ]:
from pyproj import Geod

geod = Geod(ellps="WGS84")

schema_final = {'geometry': 'Unknown',
                'properties': {'total_long_vias': 'float',
                              'area': 'float',
                              'densidad_vial': 'float'}}

with fiona.collection('../datos/densidad_vial.gpkg',
                      mode='w',
                      schema=schema_final,
                      driver='GPKG',
                      crs=fiona.crs.from_epsg(4326),
                      layer='densidad-vial') as collection, \
    fiona.open('../datos/limite_cantonal.geojson', 'r', 'GeoJSON', fiona.crs.from_epsg(4326)) as cantones, \
    fiona.open('../datos/red_vial.geojson', 'r', 'GeoJSON', fiona.crs.from_epsg(4326)) as vias:
    for canton in cantones:
        print(canton['properties']['canton'], canton['properties']['area'], geod.geometry_area_perimeter(shape(canton['geometry'])), shape(canton['geometry']).area)
        total_long_vias = 0
        for via in vias:
            # Esto calcula, para cada cantón y cada vía del país, la longitud de su intersección.
            #print(via['properties']['codigo'], via['properties']['longitud'], geod.geometry_length(shape(via['geometry'])), shape(via['geometry']).length, shape(canton['geometry']).intersection(shape(via['geometry'])).length)
            total_long_vias += geod.geometry_length(shape(canton['geometry']).intersection(shape(via['geometry']))) / 1000 # dividimos entre 1000 para pasar de m a km
        area, _ = geod.geometry_area_perimeter(shape(canton['geometry']))
        area = - area / 1000000 # viene en m^2, lo ajustamos a km^2
        densidad_vial = total_long_vias / area
        print(canton['properties']['canton'], total_long_vias, area, densidad_vial) # es un poco lento
        collection.write({
            'geometry': canton['geometry'],
            'properties': {'total_long_vias': total_long_vias,
                          'area': area,
                          'densidad_vial': densidad_vial}
        })
                

Corredores 623.61 (-623523618.9965256, 165918.45279262715) 0.051209670930605675
Corredores 495.7881008007988 623.5236189965257 0.7951392468479392
Golfito 1752.75 (-1752776731.025924, 597381.1830271684) 0.1439568240211488
Golfito 602.678204817061 1752.776731025924 0.34384197037138053
Coto Brus 944.24 (-944094347.9510432, 196748.30791147245) 0.07761079692547704
Coto Brus 747.4012383990618 944.0943479510432 0.7916594777006534
Osa 1932.7 (-1932929459.6571856, 450203.8809134355) 0.15887538961609998
Osa 678.937197582629 1932.9294596571856 0.3512477882679908
Buenos Aires 2382.94 (-2383003201.259017, 294437.550554591) 0.19600433659225217
Buenos Aires 1297.1130520556821 2383.003201259017 0.5443186359843646
Pérez Zeledón 1901.08 (-1901393494.8623161, 268165.63335798116) 0.1565004919192512
Pérez Zeledón 1429.6779268875757 1901.3934948623162 0.7519106017511129
Quepos 557.85 (-557963844.1011245, 171086.3062065104) 0.04593306239573115
Quepos 399.1546648854055 557.9638441011246 0.715377293896956
Tala